<a href="https://colab.research.google.com/github/nchanko/gsheet_rss_scraper/blob/main/Scrape_Telegram_Channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyrogram nest_asyncio

In [ ]:
import nest_asyncio
import asyncio
import pandas as pd
from pyrogram import Client
from datetime import datetime, timezone
import time

Telegram Channel Scraper
Script by - Nyein Chan Ko Ko

You need to apply api_id, api_hash from telegram .https://core.telegram.org/api/obtaining_api_id

#Scrape By Message Counts

In [ ]:
def scrape_by_message(api_id, api_hash, channel_username, limit):
    # Allow nested asyncio calls in Google Colab
    nest_asyncio.apply()

    # Define an async function to get channel messages with a limit
    async def get_channel_messages():
        async with Client('my_account', api_id=api_id, api_hash=api_hash) as app:
            channel_info = await app.get_chat(channel_username)

            messages_list = []

            # Get messages from the channel with specified limit
            print("Getting chat histories by messages.")
            async for message in app.get_chat_history(channel_info.id, limit=limit):
                messages_list.append({
                    'date': message.date,
                    'message': message.text
                })
                # print(f"{message.date}: {message.text}")
            print("Scraping completed.")

        return messages_list

    # Run the async function using asyncio
    loop = asyncio.get_event_loop()
    messages_data = loop.run_until_complete(get_channel_messages())

    return messages_data

#Scrape by Date

In [ ]:
def scrape_by_date(api_id, api_hash, channel_username, start_date, end_date):
    # Allow nested asyncio calls in Google Colab
    nest_asyncio.apply()

    # Convert start and end dates to UTC timezone
    start_date_utc = start_date.replace(tzinfo=timezone.utc)
    end_date_utc = end_date.replace(tzinfo=timezone.utc)

    # Define an async function to get channel messages within the specified period
    async def get_channel_messages_within_period():
        async with Client('my_account', api_id=api_id, api_hash=api_hash) as app:
            channel_info = await app.get_chat(channel_username)

            messages_list = []
            print("Getting data by date.")
            # Get messages from the channel
            async for message in app.get_chat_history(channel_info.id):
                # Convert message.date to UTC timezone
                message_date_utc = message.date.replace(tzinfo=timezone.utc)

                # Collect messages within the specified period
                if start_date_utc <= message_date_utc <= end_date_utc:
                    messages_list.append({
                        'date': message_date_utc,
                        'message': message.text
                    })

                # Break the loop if the message date is earlier than the start date
                if message_date_utc < start_date_utc:
                    break
            print("Scraping completed.")
            return messages_list

    # Run the async function using asyncio
    loop = asyncio.get_event_loop()
    messages_data = loop.run_until_complete(get_channel_messages_within_period())
    return messages_data

#Save to Sheet.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

def save_to_sheet(df,sheetname, sheet_address):

  gc = gspread.authorize(creds)
  wb = gc.open_by_url(sheet_address)
  sheet = wb.worksheet(sheetname)
  # get_all_values gives a list of rows.
  print("loading gsheet data..")
  data = sheet.get_all_values()

  # Assuming df is your DataFrame with Timestamp objects
  print("Preparing data to send to sheet. ")
  df_to_append = pd.DataFrame({
      'date': df['date'].astype(str),  # Convert Timestamp to string
      'message': df['message']
  })

  # Append the new data to the existing data
  print("Appending data to existing .")
  updated_data = data + df_to_append.values.tolist()

  # Update the Google Sheet with the combined data
  sheet.update('A1', updated_data)
  print("Process completed!")


#Main Function

In [ ]:
def main():

  # Replace with your actual values
  api_id = userdata.get('telegram_api_id')
  api_hash = userdata.get('telegram_api_hash')
  channel_username = 'waihin'
  limit = 20
  sheetname ='waihin_python'
  sheet_address = 'https://docs.google.com/spreadsheets/d/1PlJWvJgmsrMG0tX5bHJ0Bv8QzjQMQOlXlcV-s2kMooY/edit#gid=1486421084'


  #Shuld be year /month/ date format.
  start_date = datetime(2023, 12, 9, tzinfo=timezone.utc)
  end_date = datetime(2023, 12, 11, tzinfo=timezone.utc)

  #Choose method scrape by message or scrape by date. If scrape by  date , provide starting date and ending date.

  #messages_data = scrape_by_message(api_id,api_hash,channel_username,limit)
  messages_data = scrape_by_date(api_id, api_hash, channel_username, start_date, end_date)

  print("Creating dataframe.")
  df = pd.DataFrame(messages_data)
  # Display the DataFrame
  print(df)
  # Save to Google Sheet
  save_to_sheet(df,sheetname, sheet_address)



In [ ]:
main()